In [1]:
from openff.toolkit import Molecule, ForceField
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper
from openff.nagl_models import list_available_nagl_models

DEPRECATION: --no-python-version-warning is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to remove the flag as it's a no-op. Discussion can be found at https://github.com/pypa/pip/issues/13154


In [2]:
# OpenFF NAGL store models as PyTorch files. Right now (February 2025) we want the model ending in rc.3 ...
NAGL_MODEL = "openff-gnn-am1bcc-0.1.0-rc.3.pt"

# ... but in the future there should be improved models. We can list them here:
print(list_available_nagl_models())

[PosixPath('/Users/mattthompson/micromamba/envs/openff-nagl-test/lib/python3.12/site-packages/openff/nagl_models/models/am1bcc/openff-gnn-am1bcc-0.0.1-alpha.1.pt'), PosixPath('/Users/mattthompson/micromamba/envs/openff-nagl-test/lib/python3.12/site-packages/openff/nagl_models/models/am1bcc/openff-gnn-am1bcc-0.1.0-rc.1.pt'), PosixPath('/Users/mattthompson/micromamba/envs/openff-nagl-test/lib/python3.12/site-packages/openff/nagl_models/models/am1bcc/openff-gnn-am1bcc-0.1.0-rc.2.pt'), PosixPath('/Users/mattthompson/micromamba/envs/openff-nagl-test/lib/python3.12/site-packages/openff/nagl_models/models/am1bcc/openff-gnn-am1bcc-0.1.0-rc.3.pt')]


In [3]:
# load the molecule from SMILES - here you could also load from other formats, like an SDF file with 3D coordinates.
# For many ways to create a Molecule, see the cookbook:
# https://docs.openforcefield.org/projects/toolkit/en/stable/users/molecule_cookbook.html
aspirin = Molecule.from_smiles("CC(=O)OC1=CC=CC=C1C(=O)O")

aspirin.generate_conformers(n_conformers=1)

# visualization in notebook requires NGLView, which can be skipped
aspirin

NGLWidget()

In [4]:
# in order to assign partial charges, we need to
# 1. pass it the specific model (the name of a PyTorch file) we want to use
# 2. pass it the NAGL wrapper, which wires up the toolkit to the
#    `openff-nagl` software that loads the model and assigns charges
aspirin.assign_partial_charges(
    partial_charge_method=NAGL_MODEL,
    toolkit_registry=NAGLToolkitWrapper(),
)

# this `Molecule` object now contains partial charges as assigned by
# this particular NAGL model
print(aspirin.partial_charges)

[-0.16133040828364237 0.6449682797704425 -0.49803232295172556 -0.3440593693937574 0.16385800497872488 -0.15869387132780893 -0.08339377386229378 -0.14783590180533274 -0.07089014989989144 -0.13842942459242685 0.6621711935315814 -0.533313683101109 -0.6033047352518354 0.07867553191525596 0.07867553191525596 0.07867553191525596 0.1530339206968035 0.14112668590886251 0.14153014974934713 0.15063593643052237 0.4459328736577715] elementary_charge


In [5]:
# load Sage
sage = ForceField("openff-2.2.1.offxml")

# normally when we call `ForceField.create_interchange` or `ForceField.create_openmm_system`, the toolkit will call
# AmberTools or OEChem to assign partial charges, since that's what's in the force field file. A future OpenFF release
# which uses NAGL for charge assignment will encode this instruction in the force field file itself, but until that we
# can use the `charge_from_molecules` argument to tell it to use the charges that we just assigned# for more, see:
# https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.typing.engines.smirnoff.ForceField.html#openff.toolkit.typing.engines.smirnoff.ForceField.create_openmm_system
interchange = sage.create_interchange(
    aspirin.to_topology(),
    charge_from_molecules=[aspirin],
)

interchange.visualize() # this also visualizes the 3D structure via NGLview

NGLWidget()

In [6]:
# From here you can export to MD software (OpenMM, GROMACS, etc.). See
# https://docs.openforcefield.org/projects/interchange/en/stable/using/output.html